## Imports

In [1]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

# For drawing onto the image:
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

from imutils import paths

ModuleNotFoundError: No module named 'object_detection'

## Set paths

In [ ]:
tf.get_logger().setLevel('INFO')

BASE_PATH = "../Data/AAU_RainSnow"
EGENSEVEJ_BASE = os.path.join(BASE_PATH, 'Egensevej_copy')
EGENSEVEJ_1 = os.path.join(EGENSEVEJ_BASE, 'Egensevej-1')
EGENSEVEJ_1_TRAIN = os.path.join(EGENSEVEJ_1, 'train')

## Utilities

In [ ]:
def get_files_from_folder(path):

    files = os.listdir(path)
    return np.asarray(files)

def custom_data_split(path_to_data, path_to_test_data, train_ratio):
    get dirs    dirpath, dirs, filenames = next(os.walk(path_to_data))

    # below commented-out code is unnecessary for our folder structure

    # calculates how many train data per class
    # data_counter_per_class = np.zeros((len(dirs)))
    # for i in range(len(dirs)):
    #     path = os.path.join(path_to_data, dirs[i])
    #     files = get_files_from_folder(path)
    #     data_counter_per_class[i] = len(files)

    file_count = len(filenames)
    test_count = np.round(file_count * (1 - train_ratio))

    # transfer files
    # create dir
    if not os.path.exists(path_to_test_data):
        os.makedirs(path_to_test_data)
    files = get_files_from_folder(path_to_data)
    # moves data
    for j in range(int(test_count)):
        dst = os.path.join(path_to_test_data, files[j])
        src = os.path.join(path_to_data, files[j])
        shutil.move(src, dst)

In [ ]:
def compute_iou(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the intersection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

## Data Prep

In [ ]:
custom_data_split(EGENSEVEJ_1, EGENSEVEJ_1_TRAIN, 0.8)

## Create Model

In [ ]:
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models


model = tf.sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1])
])